## Chapter 23: Modules and Tests

This chapter covers both modules and tests.  Modules are another great way to separate code and put types and common functions together in that they will be used.

#### 23.1: The Revise package

The `Revise` package is a great package to run to build packages.  Recall that since `struct`s are immutable, if you are trying to design a type, then you can't change it once you've made it.  This makes building a type and iterating on it difficult.  The `Revise` package handles this for you.  

In [2]:
using Revise

#### 23.2: Creating a module

A `module` is a block of code that often contains:

* custom types (structs)
* functions
* other data needed by the functions and types. 

A great example is that of the Playing Cards that we've been using in a few chapters in this course.  To creating, here's a template:

```
module PlayingCards

end
```

We can then put all of the related playing card types and functions inside this module. 

Note: Julia convention is that a module name is in Pascal case.  

Instead of building this module in a jupyter cell, we'll often build it in a separate file.  This has already been done if you go to `../julia-files/PlayingCards.jl`, where `..` means up a directory.  You can open this inside jupyter. 

Then to load it, we will use the function `includet`, which is part of the `Revise` module.  The `t` stands for tracking in that the function `includet` will track changes to the module and reload as needed.

In [3]:
includet("../julia-files/PlayingCards.jl")

And since it is a module, we need to give access to it with the `using` command.  Technically, the `PlayingCards` module is now in `Main` and can be loaded with either `using Main.PlayingCards` or the shortcut: 

In [4]:
using .PlayingCards

Now we have access to all of the types and functions.

In [5]:
h = Hand("2♡,6♣,2♠,2♢,6♢")

[2♡,6♣,2♠,2♢,6♢]

In [6]:
isFullHouse(h)

true

If you notice, the module doesn't include the `runTrials` function we wrote in Chapter 15, so let's add that to the module.  (and don't forget to export it)

In [ ]:
"""
    runTrials(f::Function, trials::Integer)

For `trials` randomly selected hands, run the function `f` on each hand.  The fraction of hands where `f` is true is returned.

### Example
```julia-repl
runTrials(isFullHouse, 10_000_000)
```
"""
function runTrials(f::Function, trials::Integer)
  local deck=collect(1:52) # creates the array [1,2,3,...,52]
  local num_hands=0
  for i=1:trials
    shuffle!(deck)
    h = Hand(map(Card,deck[1:5])) # creates a hand of the first five cards of the shuffled deck
    if f(h)
      num_hands+=1
    end
  end
  num_hands/trials
end

In [8]:
runTrials(isFullHouse,10_000_000)

0.0014491

#### Documenting functions in modules

In the REPL and other Jupyter notebooks, use `?` before a type or function in Julia will show its documentation.  However, this does appear to work in VSCode.  I found work around [here](https://discourse.julialang.org/t/displaying-docstrings-in-notebook-in-vscode-errors/107627/3).  If we define the follow macro:

In [9]:
?push!

Base.Meta.ParseError: ParseError:
# Error @ /Users/pstaab/code/sci-comp-notebooks/notebooks/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y142sZmlsZQ==.jl:1:1
?push!
╙ ── not a unary operator

In [10]:
macro docVSC(fn)
  return :( display("text/markdown",Base.doc($fn)))
end

@docVSC (macro with 1 method)

Then do `@docVSC obj` will generate the documentation correctly.  For example on the built-in documentation:

In [11]:
@docVSC push!

```
push!(collection, items...) -> collection
```

Insert one or more `items` in `collection`. If `collection` is an ordered container, the items are inserted at the end (in the given order).

# Examples

```jldoctest
julia> push!([1, 2, 3], 4, 5, 6)
6-element Vector{Int64}:
 1
 2
 3
 4
 5
 6
```

If `collection` is ordered, use [`append!`](@ref) to add all the elements of another collection to it. The result of the preceding example is equivalent to `append!([1, 2, 3], [4, 5, 6])`. For `AbstractSet` objects, [`union!`](@ref) can be used instead.

See [`sizehint!`](@ref) for notes about the performance model.

See also [`pushfirst!`](@ref).


And if we define the help documentation correctly in a module, we can use it for this too:

In [13]:
@docVSC Card

```
Card(r::Int, s::Int)
Card(i::Int)
Card(str::String)
```

Create a Card object that represents a playing card with rank `r` and suit `s`.  The rank must satisfy `1<=r<=13` and the suit represents `1<=s<=4`.   In addition, one can make a Card with a single integer `n` that satifies `1<=n<=52`. Lastly, You can create a Card with a string consisting of the rank as `A,1,2,3,...,9,T,J,Q,K` and the suit ♣,♠,♡,♢.

### Examples

```julia-repl
julia> Card(10,3)
T♡

julia> Card(33)
7♢

julia> Card("J♠")
J♠
```


With the `Revise` module loaded and using `includet` on the file containing the module, you will have the ability to edit the julia file (module) and immediately changes are available without restarting the kernel.  This is very useful for creating a module.  

First, let's add a `isTwoPair` function to the module. Copy this from when we developed this in Chapter 20. 

In [14]:
two_pair = Hand("T♠,T♠,Q♡,A♠,A♢")

[T♠,T♠,Q♡,A♠,A♢]

In [18]:
isTwoPair(two_pair)

true

### Exercise

Start a Geometry module by the following steps. 
1. create a file in VScode called `Geometry.jl`.
2. Put the template `module Geometry` then `end` in the file. 
3. Add your `struct` for `Point2D` from HW #5 inside the module.
4. Add the `Base.show` method for `Point2D`
4. Save the file
5. Load the file with `includet("Geometry.jl")`
6. Create a `Point2D` inside the notebook. 

#### 23.3: Unit Tests

A unit test is a way to ensure that code is written in a robust manner.  A test is a piece of code that will determine if something is returned true. In Julia, we will use the `Test` package

In [19]:
using Test

A very simple example is if we define a variable

In [20]:
n=3

3

and then test if this is 3 using the `@test` macro.

In [21]:
@test n==3

Test Passed

We'll start by testing some of the `PlayingCards` types and function.  First, we can test if create a `Card` actually makes a card.

In [22]:
@test isa(Card(1,4),Card)

Test Passed

which says yes, running the code `Card(1,4)` creates a Card object.  We can also test the other constructors as well:

In [23]:
@test isa(Card(35),Card)

Test Passed

In [24]:
@test isa(Card("3♣"),Card)

Test Passed

It's a good idea to also make sure that the checking routines work:

In [25]:
@test isa(Card(13,5),Card)

Error During Test at /Users/pstaab/code/sci-comp-notebooks/notebooks/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X55sZmlsZQ==.jl:1
  Test threw exception
  Expression: Card(13, 5) isa Card
  ArgumentError: The suit must be an integer between 1 and 4.
  Stacktrace:
   [1] Card(r::Int64, s::Int64)
     @ Main.PlayingCards ~/code/sci-comp-notebooks/julia-files/PlayingCards.jl:41
   [2] macro expansion
     @ ~/.julia/juliaup/julia-1.11.1+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Test/src/Test.jl:676 [inlined]
   [3] top-level scope
     @ ~/code/sci-comp-notebooks/notebooks/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X55sZmlsZQ==.jl:1


Test.FallbackTestSetException: Test.FallbackTestSetException("There was an error during testing")

However, notice that there's an error that the test wasn't passed as well as the error that is throw from the constructor.  A better way to do this is to test if an error is thrown:

In [26]:
@test_throws ArgumentError Card(13,5)

Test Passed
      Thrown: ArgumentError

##### Test Sets

A nice way to collect tests together in common ways is to make a test set.  The following covers all successfully constructed Cards:

In [29]:
@testset "Legal Card Constructor" begin
        @test isa(Card(1,3),Card)
        @test isa(Card(45),Card)
        @test isa(Card("3\u2660"),Card)
        @test isa(Card("T♣"),Card)
end;

Test Summary:          | Pass  Total  Time
Legal Card Constructor |    4      4  0.0s


The idea of a test suite (many test sets that are associated with a module) is to have it on hand so when you update things, you don't break anything (have a regression). So we will put this in a file that we can just load

#### Checking for Equality

It would also be nice if the Card contructors actually generate the right card.  First put the following in the `PlayingCards.jl` file

```
import Base.==
```
(at top of the file)

and later put
```
Base.==(c1::Card, c2::Card) = c1.rank == c2.rank && c1.suit == c2.suit
```

Then we can test if a card is created correctly. 

In [31]:
@test Card("3♣") == Card(3,4)

Test Passed

### Exercise
- Construct the following hands: full house, four of a kind, 2 pair and royal flush
- create a test suite on the  hands on `isFullHouse` and another test suite for `isRoyalFlush`.
- For each, test that only the correct hand works.

In [32]:
royal_flush = Hand("T♠,J♠,Q♠,K♠,A♠")
full_house = Hand("4♣,4♢,4♠,7♣,7♢")
four_of_a_kind = Hand("2♠,5♠,5♢,5♣,5♡")
two_pair = Hand("4♣,4♢,5♠,7♣,7♢")

[4♣,4♢,5♠,7♣,7♢]

In [34]:
@testset "Testing Full House" begin
    @test !isFullHouse(royal_flush)
end;

Test Summary:      | Pass  Total  Time
Testing Full House |    1      1  0.0s


The nice thing is to put all of your tests/test suites in a single file

In [36]:
include("../julia-files/test-playing-cards.jl");

Test Summary:          | Pass  Total  Time
Legal Card Constructor |    4      4  0.0s
Test Summary:                   | Pass  Total  Time
Illegal Cards throws exceptions |    5      5  0.0s
Test Summary:          | Pass  Total  Time
Legal Hand Constructor |    3      3  0.0s
Test Summary:                  | Pass  Total  Time
Illegal Hand throws exceptions |    5      5  0.0s
Test Summary: | Pass  Total  Time
Card Tests    |    3      3  0.0s
Test Summary: | Pass  Total  Time
Full House    |    2      2  0.0s


#### 23.5: A Rootfinding Module

We will also create a Rootfinding module that has some of the functions that we have seen over this course.  They are in the `Rootfinding.jl` file in the `julia-files` directory:

In [37]:
includet("../julia-files/Rootfinding.jl")
using .Rootfinding

Take a look at that file:

In [38]:
newton(x->x^2-2,1)

The root is approximately x̂ = 1.4142135623746899
An estimate for the error is 1.5947429102833119e-12
with f(x̂) = 4.510614104447086e-12
which took 4 steps

In [39]:
newton(x->x^2+3,1, max_steps=19)

The root was not found within 19 steps.
Currently, the root is approximately x̂ = -1.0 
An estimate for the error is -2.0
with f(x̂) = 4.0


In [40]:
x = newton(x->x^2-2,1)

The root is approximately x̂ = 1.4142135623746899
An estimate for the error is 1.5947429102833119e-12
with f(x̂) = 4.510614104447086e-12
which took 4 steps

It would be good to include some tests with this module.  Let's test if the root above is actually $\sqrt{2}$.

In [41]:
@test x.root == sqrt(2)

Test Failed at /Users/pstaab/code/sci-comp-notebooks/notebooks/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_Y114sZmlsZQ==.jl:1
  Expression: x.root == sqrt(2)
   Evaluated: 1.4142135623746899 == 1.4142135623730951



Test.FallbackTestSetException: Test.FallbackTestSetException("There was an error during testing")

And this fails because the two are not exactly equal.  Recall that floating point numbers are equal only all bits are equal.  Instead we will use the `isapprox` function which tests for equality within a tolerance (a bit complicated), see `?isapprox`.

In [42]:
@test isapprox(x.root,sqrt(2))

Test Passed

In [43]:
@docVSC isapprox

```
isapprox(x, y; atol::Real=0, rtol::Real=atol>0 ? 0 : √eps, nans::Bool=false[, norm::Function])
```

Inexact equality comparison. Two numbers compare equal if their relative distance *or* their absolute distance is within tolerance bounds: `isapprox` returns `true` if `norm(x-y) <= max(atol, rtol*max(norm(x), norm(y)))`. The default `atol` (absolute tolerance) is zero and the default `rtol` (relative tolerance) depends on the types of `x` and `y`. The keyword argument `nans` determines whether or not NaN values are considered equal (defaults to false).

For real or complex floating-point values, if an `atol > 0` is not specified, `rtol` defaults to the square root of [`eps`](@ref) of the type of `x` or `y`, whichever is bigger (least precise). This corresponds to requiring equality of about half of the significant digits. Otherwise, e.g. for integer arguments or if an `atol > 0` is supplied, `rtol` defaults to zero.

The `norm` keyword defaults to `abs` for numeric `(x,y)` and to `LinearAlgebra.norm` for arrays (where an alternative `norm` choice is sometimes useful). When `x` and `y` are arrays, if `norm(x-y)` is not finite (i.e. `±Inf` or `NaN`), the comparison falls back to checking whether all elements of `x` and `y` are approximately equal component-wise.

The binary operator `≈` is equivalent to `isapprox` with the default arguments, and `x ≉ y` is equivalent to `!isapprox(x,y)`.

Note that `x ≈ 0` (i.e., comparing to zero with the default tolerances) is equivalent to `x == 0` since the default `atol` is `0`.  In such cases, you should either supply an appropriate `atol` (or use `norm(x) ≤ atol`) or rearrange your code (e.g. use `x ≈ y` rather than `x - y ≈ 0`).   It is not possible to pick a nonzero `atol` automatically because it depends on the overall scaling (the "units") of your problem: for example, in `x - y ≈ 0`, `atol=1e-9` is an absurdly small tolerance if `x` is the [radius of the Earth](https://en.wikipedia.org/wiki/Earth_radius) in meters, but an absurdly large tolerance if `x` is the [radius of a Hydrogen atom](https://en.wikipedia.org/wiki/Bohr_radius) in meters.

!!! compat "Julia 1.6"
    Passing the `norm` keyword argument when comparing numeric (non-array) arguments requires Julia 1.6 or later.


# Examples

```jldoctest
julia> isapprox(0.1, 0.15; atol=0.05)
true

julia> isapprox(0.1, 0.15; rtol=0.34)
true

julia> isapprox(0.1, 0.15; rtol=0.33)
false

julia> 0.1 + 1e-10 ≈ 0.1
true

julia> 1e-10 ≈ 0
false

julia> isapprox(1e-10, 0, atol=1e-8)
true

julia> isapprox([10.0^9, 1.0], [10.0^9, 2.0]) # using `norm`
true
```

```
isapprox(x; kwargs...) / ≈(x; kwargs...)
```

Create a function that compares its argument to `x` using `≈`, i.e. a function equivalent to `y -> y ≈ x`.

The keyword arguments supported here are the same as those in the 2-argument `isapprox`.

!!! compat "Julia 1.5"
    This method requires Julia 1.5 or later.



In [44]:
@test x.root ≈ sqrt(2)

Test Passed

In [45]:
x = newton(x->x^2+1,2)

The root was not found within 10 steps.
Currently, the root is approximately x̂ = 2.4008803928468465 
An estimate for the error is 1.4086971347905715
with f(x̂) = 6.764226660756428


In [46]:
@test !x.converged

Test Passed

In [47]:
@testset "function with no root" begin
  val = newton(x->x^2+1,2)
  @test !val.converged
  @test val.num_steps == val.max_steps
 end;

Test Summary:         | Pass  Total  Time
function with no root |    2      2  0.0s


### 23.6 Test Driven Development

The current thinking on developing code (either in Scientific Computing or other fields) is that of _Test Driven Development_ or TDD.  This basically flips the order of development.  One using TDD would first start with writing all of the test suites.  That is what you want everything to return.  After satisfied with this, you then write all of the functions and continually test while developing. 

### Example

If we would have used TDD to develop the Playing card code, we could write a function that goes through all 5-card hands (this isn't too bad to write) and for each hand type, determine the total number of hands (this is listed in the wikipedia article).  Then you can write all testing for types until getting the desired results. 